In [ ]:
import pandas as pd
import json
import os
import numpy as np

: 

In [ ]:
import time
from datetime import timedelta

In [56]:
start=time.time()

In [57]:
elapsed=time.time()-start

In [58]:
str(timedelta(seconds=int(elapsed)))

'0:00:19'

In [59]:
timestamp = str(int(time.time()))

In [60]:
timestamp

'1669749877'

In [61]:
!pwd

/home/na/codes/notebook_search_docker/notebookcrawler


In [66]:
file_name = '/home/na/codes/notebook_search_docker/notebookcrawler.csv'

In [73]:
timestamp = int(time.time())
names = os.path.basename(file_name).split('.')
stamped_name = names[0]+'_'+str(timestamp)+'.'+names[1]
stamped_file_name = os.path.join(os.path.dirname(file_name), stamped_name)

In [1]:
import time
import pandas as pd

from pymongo import MongoClient

client = MongoClient('localhost', 27017) # change the ip and port to your mongo database's
db = client['kagglecrawler']
search_log_coll = db['search_log']
download_log_coll = db['download_log']
raw_notebook_coll = db['raw_notebooks']
task_log = db['task_log']


ModuleNotFoundError: No module named 'pymongo'